✅ Ball possession
✅ Pass detection
✅ Team heatmap
✅ Tactical lines
✅ FPS boost (detect every N frames)
✅ Full video output (VS Code friendly)

In [ ]:
# ! pip install ultralytics supervision opencv-python numpy tqdm sports-analytics


### 0

In [ ]:
import cv2
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import supervision as sv
from ultralytics import YOLO
from sports.common.team import TeamClassifier

# ==============================
# CONFIG
# ==============================
VIDEO_PATH = "input.mp4"
OUTPUT_PATH = "final_analytics.mp4"
MODEL_PATH = "foatball350.pt"

WIDTH, HEIGHT = 1280, 720
CONF = 0.5
DETECT_EVERY = 3
BALL_DIST_TH = 80
PASS_TIME_TH = 1.0

# ==============================
# LOAD MODEL
# ==============================
model = YOLO(MODEL_PATH)

# ==============================
# VIDEO INFO
# ==============================
cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS) or 30
cap.release()
frame_time = 1 / fps

# ==============================
# UTIL
# ==============================
def center(box):
    x1, y1, x2, y2 = box
    return np.array([(x1+x2)/2, (y1+y2)/2])

# ==============================
# TEAM TRAINING
# ==============================
def extract_crops(video, stride=20, max_crops=120):
    crops = []
    cap = cv2.VideoCapture(video)
    idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or len(crops) >= max_crops:
            break
        if idx % stride == 0:
            frame = cv2.resize(frame, (WIDTH, HEIGHT))
            r = model.track(frame, persist=True, conf=0.3)[0]
            det = sv.Detections.from_ultralytics(r)
            players = det[det.class_id == 2]
            for box in players.xyxy:
                x1, y1, x2, y2 = map(int, box)
                crop = frame[y1:y2, x1:x2]
                if crop.size > 0:
                    crops.append(crop)
        idx += 1
    cap.release()
    return crops

print("[INFO] Training Team Classifier...")
crops = extract_crops(VIDEO_PATH)
team_clf = TeamClassifier(device="cuda" if cv2.cuda.getCudaEnabledDeviceCount() else "cpu")
team_clf.fit(crops)

# ==============================
# ANALYTICS STORAGE
# ==============================
possession = defaultdict(float)
last_owner = None
last_owner_time = 0
passes = []

heatmap = {
    0: np.zeros((HEIGHT, WIDTH)),
    1: np.zeros((HEIGHT, WIDTH))
}

# ==============================
# ANNOTATORS
# ==============================
palette = sv.ColorPalette.from_hex(["#00BFFF", "#FF1493"])
ellipse = sv.EllipseAnnotator(color=palette)
triangle = sv.TriangleAnnotator(color=sv.Color.YELLOW)
tracker = sv.ByteTrack()

# ==============================
# VIDEO WRITER
# ==============================
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (WIDTH, HEIGHT))

# ==============================
# MAIN LOOP
# ==============================
cached_det = None
frames = sv.get_video_frames_generator(VIDEO_PATH)

for idx, frame in enumerate(tqdm(frames)):
    frame = cv2.resize(frame, (WIDTH, HEIGHT))
    scene = frame.copy()

    # ---- Detection (FPS Boost)
    if idx % DETECT_EVERY == 0:
        r = model.track(frame, persist=True, conf=CONF)[0]
        cached_det = sv.Detections.from_ultralytics(r)

    if cached_det is None:
        out.write(scene)
        continue

    det = cached_det

    ball = det[det.class_id == 0]
    players = det[det.class_id == 2].with_nms(0.3)
    players = tracker.update_with_detections(players)

    # ---- Team Classification
    crops, valid = [], []
    for i, box in enumerate(players.xyxy):
        x1, y1, x2, y2 = map(int, box)
        crop = frame[y1:y2, x1:x2]
        if crop.size > 0:
            crops.append(crop)
            valid.append(i)

    team_ids = [0] * len(players)
    if crops:
        preds = team_clf.predict(crops)
        for i, t in zip(valid, preds):
            team_ids[i] = t
    players.class_id = team_ids

    # ---- Heatmap
    for box, tid in zip(players.xyxy, team_ids):
        cx, cy = map(int, center(box))
        if 0 <= cx < WIDTH and 0 <= cy < HEIGHT:
            heatmap[tid][cy, cx] += 1

    # ---- Ball Possession
    owner = None
    if len(ball) and len(players):
        bc = center(ball.xyxy[0])
        min_d = 1e9
        for box, pid in zip(players.xyxy, players.tracker_id):
            if pid is None:
                continue
            d = np.linalg.norm(center(box) - bc)
            if d < min_d and d < BALL_DIST_TH:
                min_d = d
                owner = pid

    if owner:
        possession[owner] += frame_time
        if last_owner and owner != last_owner:
            if (idx / fps - last_owner_time) < PASS_TIME_TH:
                passes.append((last_owner, owner))
        last_owner = owner
        last_owner_time = idx / fps

    # ---- Tactical Lines
    for tid in [0, 1]:
        xs = [center(b)[0] for b, t in zip(players.xyxy, team_ids) if t == tid]
        if xs:
            x = int(np.mean(xs))
            cv2.line(scene, (x, 0), (x, HEIGHT), palette.by_idx(tid).as_bgr(), 2)

    # ---- Draw
    scene = ellipse.annotate(scene, players)
    if len(ball):
        scene = triangle.annotate(scene, ball)

    # ---- Overlay Stats
    y = 30
    for pid, t in list(possession.items())[:3]:
        cv2.putText(scene, f"P{pid}: {t:.1f}s", (20, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
        y += 25

    out.write(scene)

out.release()

print("\n[DONE]")
print("Top Passes:", passes[:5])
print("Top Possession:", sorted(possession.items(), key=lambda x: -x[1])[:5])
print(f"Saved video: {OUTPUT_PATH}")


## 01

football_analysis/

├── main.py              # প্রধান ফাইল (চালু করবেন এটা)

├── config.py            # সেটিংস এবং থ্রেশহোল্ড

├── detector.py          # বল এবং খেলোয়াড় ডিটেক্ট

├── analyzer.py          # পাস এবং মালিকানা বিশ্লেষণ

├── visualizer.py        # ভিজ্যুয়ালাইজেশন

└── utils.py             # সাহায্যকারী ফাংশন

📄 ১. config.py

In [ ]:
"""
config.py - সব সেটিংস এবং থ্রেশহোল্ড এখানে
"""

# থ্রেশহোল্ড সেটিংস
PASS_TIME_THRESHOLD = 1.0      # পাস হিসেবে গণ্য করার সর্বোচ্চ সময়
QUICK_PASS_THRESHOLD = 0.5     # দ্রুত পাসের থ্রেশহোল্ড
DISTANCE_THRESHOLD = 200       # দীর্ঘ পাসের দূরত্ব থ্রেশহোল্ড (পিক্সেল)
BALL_DISTANCE_THRESHOLD = 80   # বলের মালিক হবার সর্বোচ্চ দূরত্ব

# দলের খেলোয়াড় আইডি
TEAM_A_PLAYERS = [7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27]
TEAM_B_PLAYERS = [10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]

# কালার কোড (BGR ফরম্যাট)
COLORS = {
    'TEAM_A': (0, 0, 255),        # লাল
    'TEAM_B': (255, 0, 0),        # নীল
    'QUICK_PASS': (0, 255, 0),    # সবুজ
    'NORMAL_PASS': (255, 255, 0), # হলুদ
    'SUCCESSFUL': (0, 255, 0),    # সবুজ
    'FAILED': (0, 0, 255),        # লাল
    'INTERCEPTION': (0, 165, 255) # কমলা
}

# ভিডিও সেটিংস
DEFAULT_FPS = 30
FRAME_SKIP = 1  # প্রতিটি ফ্রেম প্রসেস করতে চাইলে 1

📄 ২. utils.py 

In [ ]:
"""
utils.py - সাধারণ সাহায্যকারী ফাংশন
"""
import numpy as np

def box_center(xyxy):
    """
    বাউন্ডিং বক্সের কেন্দ্র বের করে
    
    Args:
        xyxy: [x1, y1, x2, y2] ফরম্যাটে বক্স
    
    Returns:
        numpy array: [center_x, center_y]
    """
    x1, y1, x2, y2 = xyxy
    return np.array([(x1 + x2) / 2, (y1 + y2) / 2])


def calculate_distance(point1, point2):
    """
    দুটি পয়েন্টের মধ্যে ইউক্লিডীয় দূরত্ব বের করে
    
    Args:
        point1: প্রথম পয়েন্ট [x, y]
        point2: দ্বিতীয় পয়েন্ট [x, y]
    
    Returns:
        float: দূরত্ব (পিক্সেল)
    """
    return np.linalg.norm(np.array(point1) - np.array(point2))


def calculate_pass_type(duration, distance):
    """
    পাসের ধরন নির্ধারণ করে
    
    Args:
        duration: পাসের সময় (সেকেন্ড)
        distance: পাসের দূরত্ব (পিক্সেল)
    
    Returns:
        str: পাসের ধরন
    """
    from config import QUICK_PASS_THRESHOLD, DISTANCE_THRESHOLD
    
    if duration < QUICK_PASS_THRESHOLD:
        if distance is not None and distance > DISTANCE_THRESHOLD:
            return "QUICK_LONG_PASS"
        else:
            return "QUICK_SHORT_PASS"
    else:
        if distance is not None and distance > DISTANCE_THRESHOLD:
            return "NORMAL_LONG_PASS"
        else:
            return "NORMAL_SHORT_PASS"


def print_pass_info(pass_info):
    """
    পাসের তথ্য সুন্দরভাবে প্রিন্ট করে
    """
    colors = {
        "QUICK_LONG_PASS": "🟢",
        "QUICK_SHORT_PASS": "🟡",
        "NORMAL_LONG_PASS": "🔵",
        "NORMAL_SHORT_PASS": "⚪"
    }
    
    color = colors.get(pass_info['pass_type'], "⚪")
    distance_str = f"{pass_info['distance']:.1f}px" if pass_info['distance'] else "N/A"
    
    print(f"{color} PASS: Player {pass_info['from_player']} → Player {pass_info['to_player']}")
    print(f"   Type: {pass_info['pass_type']}")
    print(f"   Time: {pass_info['time']:.3f}s | Distance: {distance_str}")
    print(f"   Frame Time: {pass_info['frame_time']:.2f}s")
    print("-" * 40)

📄 ৩. detector.py

In [ ]:
"""
detector.py - বল এবং খেলোয়াড় ডিটেক্ট করে
"""
import numpy as np
from config import BALL_DISTANCE_THRESHOLD
from utils import box_center, calculate_distance


class BallDetector:
    """বল ডিটেক্ট এবং মালিক নির্ধারণ করে"""
    
    def __init__(self):
        self.ball_threshold = BALL_DISTANCE_THRESHOLD
    
    def get_current_owner(self, detection_results):
        """
        বর্তমান ফ্রেমের বল মালিক নির্ধারণ করে
        
        Args:
            detection_results: ডিটেকশন রেজাল্ট ডিকশনারি
        
        Returns:
            tuple: (owner_id, owner_position) বা (None, None)
        """
        # ডিটেকশন রেজাল্ট থেকে বল এবং খেলোয়াড় আলাদা করুন
        ball_boxes = detection_results.get('ball', [])
        player_boxes = detection_results.get('players', {}).get('boxes', [])
        player_ids = detection_results.get('players', {}).get('ids', [])
        
        # যদি বল না পাওয়া যায় বা খেলোয়াড় না পাওয়া যায়
        if not ball_boxes or not player_boxes:
            return None, None
        
        # বলের প্রথম বক্স নিন
        ball_box = ball_boxes[0]
        ball_center = box_center(ball_box)
        
        # সবচেয়ে কাছের খেলোয়াড় খুঁজুন
        min_distance = float('inf')
        owner_id = None
        owner_position = None
        
        for box, pid in zip(player_boxes, player_ids):
            if pid is None:
                continue
            
            player_center = box_center(box)
            distance = calculate_distance(ball_center, player_center)
            
            # থ্রেশহোল্ডের মধ্যে এবং সবচেয়ে কাছের হলে
            if distance < min_distance and distance < self.ball_threshold:
                min_distance = distance
                owner_id = pid
                owner_position = player_center
        
        return owner_id, owner_position
    
    
    def simulate_detection(self, frame_idx):
        """
        টেস্টিং এর জন্য ডামি ডিটেকশন ডেটা জেনারেট করে
        """
        # ডামি ডেটা - বাস্তবে YOLO/Detectron2 ব্যবহার করবেন
        
        # বল অবস্থান (ডামি)
        ball_x = 300 + 5 * np.sin(frame_idx * 0.1)
        ball_y = 150 + 3 * np.cos(frame_idx * 0.05)
        ball_box = [[ball_x-10, ball_y-10, ball_x+10, ball_y+10]]
        
        # খেলোয়াড় ডেটা (ডামি)
        player_boxes = []
        player_ids = []
        
        # 4 জন খেলোয়াড়
        for i in range(4):
            player_id = 7 if i < 2 else 10  # 2 জন দল A, 2 জন দল B
            offset_x = i * 50
            player_box = [
                ball_x - 30 + offset_x,
                ball_y - 40 + (i % 2) * 30,
                ball_x + 30 + offset_x,
                ball_y + 40 + (i % 2) * 30
            ]
            player_boxes.append(player_box)
            player_ids.append(player_id)
        
        detection_results = {
            'ball': ball_box,
            'players': {
                'boxes': player_boxes,
                'ids': player_ids
            }
        }
        
        return detection_results

📄 ৪. analyzer.py

In [ ]:
"""
analyzer.py - পাস এবং মালিকানা বিশ্লেষণ করে
"""
from config import PASS_TIME_THRESHOLD, TEAM_A_PLAYERS, TEAM_B_PLAYERS
from utils import calculate_distance, calculate_pass_type, print_pass_info


class PassAnalyzer:
    """পাস বিশ্লেষণ করে"""
    
    def __init__(self, fps=30):
        self.fps = fps
        
        # ট্র্যাকিং ভেরিয়েবল
        self.last_owner = None
        self.last_owner_position = None
        self.last_change_time = None
        
        # পরিসংখ্যান
        self.pass_events = []
        self.possession_stats = {
            'team_a': 0,
            'team_b': 0,
            'no_owner': 0
        }
    
    def detect_pass(self, owner_id, owner_position, frame_idx):
        """
        একটি ফ্রেমের জন্য পাস ডিটেক্ট করে
        
        Args:
            owner_id: বর্তমান মালিক আইডি
            owner_position: মালিকের অবস্থান
            frame_idx: ফ্রেম নম্বর
        
        Returns:
            dict: পাস ইভেন্ট বা None
        """
        current_time = frame_idx / self.fps
        
        if owner_id is None:
            if self.last_owner is not None:
                self.possession_stats['no_owner'] += 1
            return None
        
        # প্রথম বার বল মালিক পাওয়া গেছে
        if self.last_owner is None:
            self.last_owner = owner_id
            self.last_owner_position = owner_position
            self.last_change_time = current_time
            return None
        
        # যদি মালিক বদলায়
        if owner_id != self.last_owner:
            dt = current_time - self.last_change_time
            
            # পাস হয়েছে কিনা চেক
            if dt < PASS_TIME_THRESHOLD:
                # পাসের দূরত্ব ক্যালকুলেট
                pass_distance = None
                if self.last_owner_position is not None and owner_position is not None:
                    pass_distance = calculate_distance(
                        self.last_owner_position, owner_position
                    )
                
                # পাস ইভেন্ট তৈরি
                pass_event = {
                    'from_player': self.last_owner,
                    'to_player': owner_id,
                    'from_position': self.last_owner_position,
                    'to_position': owner_position,
                    'time': dt,
                    'distance': pass_distance,
                    'pass_type': calculate_pass_type(dt, pass_distance),
                    'frame_time': current_time,
                    'successful': dt < 0.5  # ডামি সফলতা চেক
                }
                
                # আউটপুট প্রিন্ট
                print_pass_info(pass_event)
                
                # পাস ইভেন্ট স্টোর
                self.pass_events.append(pass_event)
                
                # পরিসংখ্যান আপডেট
                self.update_possession_stats(owner_id)
                
                # ভেরিয়েবল আপডেট
                self.last_owner = owner_id
                self.last_owner_position = owner_position
                self.last_change_time = current_time
                
                return pass_event
        
        # মালিক বদল না হলে শুধু পজিশন আপডেট
        if owner_id == self.last_owner:
            self.last_owner_position = owner_position
        
        # পরিসংখ্যান আপডেট
        self.update_possession_stats(owner_id)
        
        return None
    
    def update_possession_stats(self, owner_id):
        """মালিকানা পরিসংখ্যান আপডেট"""
        if owner_id in TEAM_A_PLAYERS:
            self.possession_stats['team_a'] += 1
        elif owner_id in TEAM_B_PLAYERS:
            self.possession_stats['team_b'] += 1
        else:
            self.possession_stats['no_owner'] += 1
    
    def get_stats(self):
        """বর্তমান পরিসংখ্যান রিটার্ন করে"""
        total_frames = sum(self.possession_stats.values())
        
        if total_frames == 0:
            return {
                'team_a_percent': 0,
                'team_b_percent': 0,
                'total_passes': len(self.pass_events),
                'successful_passes': sum(1 for p in self.pass_events if p.get('successful')),
                'avg_pass_time': 0
            }
        
        team_a_percent = (self.possession_stats['team_a'] / total_frames) * 100
        team_b_percent = (self.possession_stats['team_b'] / total_frames) * 100
        
        avg_pass_time = 0
        if self.pass_events:
            avg_pass_time = sum(p['time'] for p in self.pass_events) / len(self.pass_events)
        
        successful_passes = sum(1 for p in self.pass_events if p.get('successful'))
        
        return {
            'team_a_percent': team_a_percent,
            'team_b_percent': team_b_percent,
            'total_passes': len(self.pass_events),
            'successful_passes': successful_passes,
            'avg_pass_time': avg_pass_time,
            'possession_team_a': self.possession_stats['team_a'],
            'possession_team_b': self.possession_stats['team_b']
        }
    
    def print_summary(self):
        """বিশ্লেষণ সারাংশ প্রিন্ট করে"""
        stats = self.get_stats()
        
        print("\n" + "="*50)
        print("PASS ANALYSIS SUMMARY")
        print("="*50)
        
        print(f"\n📊 Possession:")
        print(f"  Team A: {stats['team_a_percent']:.1f}% ({stats['possession_team_a']} frames)")
        print(f"  Team B: {stats['team_b_percent']:.1f}% ({stats['possession_team_b']} frames)")
        
        print(f"\n🎯 Passing Statistics:")
        print(f"  Total Passes: {stats['total_passes']}")
        print(f"  Successful Passes: {stats['successful_passes']}")
        
        if stats['total_passes'] > 0:
            success_rate = (stats['successful_passes'] / stats['total_passes']) * 100
            print(f"  Success Rate: {success_rate:.1f}%")
            print(f"  Average Pass Time: {stats['avg_pass_time']:.3f}s")
        
        print("="*50)

📄 ৫. visualizer.py

In [ ]:
"""
visualizer.py - ভিজ্যুয়ালাইজেশন ফাংশন
"""
import cv2
import numpy as np
from config import COLORS


class FootballVisualizer:
    """ফুটবল বিশ্লেষণ ভিজ্যুয়ালাইজেশন"""
    
    def __init__(self):
        self.pass_history = []
    
    def draw_player(self, frame, player_id, position, team, has_ball=False):
        """
        খেলোয়াড় আঁকে
        
        Args:
            frame: ভিডিও ফ্রেম
            player_id: খেলোয়াড় আইডি
            position: অবস্থান (x, y)
            team: দল ('A' বা 'B')
            has_ball: বল আছে কিনা
        
        Returns:
            numpy array: আপডেটেড ফ্রেম
        """
        if position is None:
            return frame
        
        x, y = int(position[0]), int(position[1])
        
        # রং নির্ধারণ
        if team == 'A':
            color = COLORS['TEAM_A']  # লাল
        else:
            color = COLORS['TEAM_B']  # নীল
        
        # বৃত্ত আঁকা
        radius = 25 if has_ball else 20
        thickness = 3 if has_ball else 2
        
        cv2.circle(frame, (x, y), radius, color, thickness)
        
        # খেলোয়াড় নম্বর
        cv2.putText(
            frame, str(player_id), 
            (x - 10, y + 5),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2
        )
        
        # বল থাকলে ছোট বৃত্ত
        if has_ball:
            cv2.circle(frame, (x, y - radius - 5), 5, (0, 255, 255), -1)
        
        return frame
    
    def draw_pass_arrow(self, frame, from_pos, to_pos, pass_type, duration):
        """
        পাসের তীর আঁকে
        
        Args:
            frame: ভিডিও ফ্রেম
            from_pos: পাস শুরু অবস্থান
            to_pos: পাস শেষ অবস্থান
            pass_type: পাসের ধরন
            duration: পাসের সময়
        
        Returns:
            numpy array: আপডেটেড ফ্রেম
        """
        if from_pos is None or to_pos is None:
            return frame
        
        # কালার নির্ধারণ
        if duration < 0.5:
            color = COLORS['QUICK_PASS']
            thickness = 3
        else:
            color = COLORS['NORMAL_PASS']
            thickness = 2
        
        # তীর আঁকা
        cv2.arrowedLine(
            frame,
            tuple(map(int, from_pos)),
            tuple(map(int, to_pos)),
            color,
            thickness,
            tipLength=0.2
        )
        
        # পাসের সময় দেখানো
        mid_x = int((from_pos[0] + to_pos[0]) / 2)
        mid_y = int((from_pos[1] + to_pos[1]) / 2)
        
        cv2.putText(
            frame, f"{duration:.2f}s",
            (mid_x - 20, mid_y - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1
        )
        
        return frame
    
    def draw_stats_panel(self, frame, stats):
        """
        পরিসংখ্যান প্যানেল আঁকে
        
        Args:
            frame: ভিডিও ফ্রেম
            stats: পরিসংখ্যান ডিকশনারি
        
        Returns:
            numpy array: আপডেটেড ফ্রেম
        """
        height, width = frame.shape[:2]
        
        # আধা-স্বচ্ছ প্যানেল তৈরি
        overlay = frame.copy()
        cv2.rectangle(overlay, (10, 10), (300, 180), (0, 0, 0), -1)
        frame = cv2.addWeighted(overlay, 0.6, frame, 0.4, 0)
        
        # টেক্সট লিস্ট
        stats_text = [
            "⚽ LIVE STATS",
            f"Team A: {stats.get('team_a_percent', 0):.1f}%",
            f"Team B: {stats.get('team_b_percent', 0):.1f}%",
            f"Passes: {stats.get('total_passes', 0)}",
            f"Successful: {stats.get('successful_passes', 0)}",
            f"Avg Pass: {stats.get('avg_pass_time', 0):.2f}s"
        ]
        
        # টেক্সট আঁকা
        y_offset = 40
        line_height = 25
        
        for i, text in enumerate(stats_text):
            color = (0, 255, 155) if i == 0 else (200, 200, 200)
            font_size = 0.6 if i == 0 else 0.5
            
            cv2.putText(
                frame, text,
                (20, y_offset + i * line_height),
                cv2.FONT_HERSHEY_SIMPLEX, font_size, color, 1
            )
        
        return frame
    
    def draw_pass_history(self, frame, pass_history):
        """
        পাসের ইতিহাস আঁকে
        
        Args:
            frame: ভিডিও ফ্রেম
            pass_history: পাস ইভেন্টের তালিকা
        
        Returns:
            numpy array: আপডেটেড ফ্রেম
        """
        height, width = frame.shape[:2]
        
        # প্যানেল আঁকা
        panel_x = width - 250
        cv2.rectangle(frame, (panel_x, 10), (width - 10, 180), (0, 0, 0), -1)
        
        # টাইটেল
        cv2.putText(
            frame, "RECENT PASSES",
            (panel_x + 10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1
        )
        
        # প্রতিটি পাস দেখানো
        y_offset = 60
        recent_passes = pass_history[-5:]  # সর্বশেষ ৫টি পাস
        
        for i, pass_event in enumerate(recent_passes):
            from_player = pass_event.get('from_player', '?')
            to_player = pass_event.get('to_player', '?')
            duration = pass_event.get('time', 0)
            
            # কালার নির্ধারণ
            if duration < 0.5:
                color = (0, 255, 0)  # সবুজ
            elif duration < 1.0:
                color = (255, 255, 0)  # হলুদ
            else:
                color = (255, 0, 0)  # লাল
            
            text = f"{from_player} → {to_player} ({duration:.2f}s)"
            
            cv2.putText(
                frame, text,
                (panel_x + 10, y_offset + i * 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1
            )
        
        return frame

📄 ৬. main.py

In [ ]:
"""
main.py - প্রধান ফাইল, এইটা চালু করবেন
"""
import cv2
import numpy as np
from config import DEFAULT_FPS, TEAM_A_PLAYERS, TEAM_B_PLAYERS
from detector import BallDetector
from analyzer import PassAnalyzer
from visualizer import FootballVisualizer


def main():
    """প্রধান ফাংশন"""
    print("="*60)
    print("FOOTBALL PASS ANALYSIS SYSTEM")
    print("="*60)
    
    # ভিডিও ফাইল পাথ
    video_path = "football_match.mp4"  # আপনার ভিডিও ফাইল পাথ
    
    # ভিডিও ক্যাপচার
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        print("Using dummy video feed instead...")
        # ডামি ভিডিও তৈরি (যদি ভিডিও না থাকে)
        cap = create_dummy_video()
    
    # FPS পাওয়া
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    if fps == 0:
        fps = DEFAULT_FPS
    
    print(f"Video FPS: {fps}")
    print(f"Team A Players: {TEAM_A_PLAYERS}")
    print(f"Team B Players: {TEAM_B_PLAYERS}")
    print("-"*60)
    
    # সিস্টেম কম্পোনেন্টস তৈরি
    detector = BallDetector()
    analyzer = PassAnalyzer(fps=fps)
    visualizer = FootballVisualizer()
    
    # প্রধান লুপ
    frame_idx = 0
    print("Starting analysis... Press 'q' to quit")
    
    while True:
        # ফ্রেম পড়া
        ret, frame = cap.read()
        if not ret:
            print("\nVideo ended or cannot read frame.")
            break
        
        # ডিটেকশন (ডামি বা আসল)
        detection_results = detector.simulate_detection(frame_idx)
        
        # বল মালিক নির্ধারণ
        owner_id, owner_position = detector.get_current_owner(detection_results)
        
        # দল নির্ধারণ
        team = 'A' if owner_id in TEAM_A_PLAYERS else 'B' if owner_id in TEAM_B_PLAYERS else None
        
        # পাস ডিটেক্ট
        pass_event = analyzer.detect_pass(owner_id, owner_position, frame_idx)
        
        # পরিসংখ্যান পাওয়া
        current_stats = analyzer.get_stats()
        
        # ভিজ্যুয়ালাইজেশন ---------------------------------
        
        # ১. খেলোয়াড় আঁকা (ডামি - আসলে আপনার ডিটেকশন থেকে আসবে)
        dummy_players = [
            {'id': 7, 'position': (200, 150), 'team': 'A'},
            {'id': 10, 'position': (300, 160), 'team': 'B'},
            {'id': 9, 'position': (250, 200), 'team': 'A'},
            {'id': 11, 'position': (350, 180), 'team': 'B'},
        ]
        
        for player in dummy_players:
            has_ball = (player['id'] == owner_id)
            frame = visualizer.draw_player(
                frame, player['id'], player['position'], 
                player['team'], has_ball
            )
        
        # ২. বল আঁকা (যদি মালিক থাকে)
        if owner_position is not None:
            cv2.circle(frame, 
                      (int(owner_position[0]), int(owner_position[1])), 
                      8, (0, 255, 255), -1)
        
        # ৩. পাস আঁকা
        if pass_event:
            frame = visualizer.draw_pass_arrow(
                frame,
                pass_event['from_position'],
                pass_event['to_position'],
                pass_event['pass_type'],
                pass_event['time']
            )
            
            # সফল পাস হাইলাইট
            if pass_event.get('successful'):
                cv2.putText(frame, "SUCCESSFUL PASS!", 
                           (frame.shape[1]//2 - 100, 50),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # ৪. পরিসংখ্যান প্যানেল
        frame = visualizer.draw_stats_panel(frame, current_stats)
        
        # ৫. পাস ইতিহাস
        frame = visualizer.draw_pass_history(frame, analyzer.pass_events)
        
        # ৬. বর্তমান মালিক দেখানো
        if owner_id is not None:
            cv2.putText(frame, f"Ball Owner: Player {owner_id} (Team {team})", 
                       (50, frame.shape[0] - 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # ৭. ফ্রেম নম্বর দেখানো
        cv2.putText(frame, f"Frame: {frame_idx}", 
                   (frame.shape[1] - 150, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        
        # ৮. ফ্রেম দেখানো
        cv2.imshow('Football Pass Analysis', frame)
        
        # কী প্রেস চেক
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            print("\nUser pressed 'q'. Stopping analysis...")
            break
        elif key == ord('p'):
            print("\nPaused. Press any key to continue...")
            cv2.waitKey(0)
        
        frame_idx += 1
        
        # প্রতি 100 ফ্রেমে প্রগ্রেস দেখানো
        if frame_idx % 100 == 0:
            print(f"Processed {frame_idx} frames...")
    
    # রিসোর্স রিলিজ
    cap.release()
    cv2.destroyAllWindows()
    
    # ফাইনাল রিপোর্ট
    print("\n" + "="*60)
    print("ANALYSIS COMPLETE!")
    print("="*60)
    
    analyzer.print_summary()
    
    print(f"\nTotal frames analyzed: {frame_idx}")
    print(f"Total time: {frame_idx/fps:.2f} seconds")
    print("="*60)


def create_dummy_video():
    """
    ডামি ভিডিও তৈরি করে (যদি ভিডিও ফাইল না থাকে)
    """
    print("Creating dummy video for testing...")
    
    # ডামি ভিডিও ক্যাপচার অবজেক্ট
    class DummyVideoCapture:
        def __init__(self):
            self.width = 640
            self.height = 480
            self.frame_count = 0
            
        def read(self):
            # ডামি ফ্রেম তৈরি
            frame = np.zeros((self.height, self.width, 3), dtype=np.uint8)
            
            # কিছু টেক্সট যোগ
            cv2.putText(frame, "DUMMY FOOTBALL VIDEO", 
                       (self.width//2 - 150, self.height//2 - 50),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            
            cv2.putText(frame, f"Frame: {self.frame_count}", 
                       (self.width//2 - 100, self.height//2),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            
            cv2.putText(frame, "Press 'q' to quit", 
                       (self.width//2 - 100, self.height//2 + 50),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            
            self.frame_count += 1
            return True, frame
        
        def get(self, prop_id):
            if prop_id == cv2.CAP_PROP_FPS:
                return 30
            elif prop_id == cv2.CAP_PROP_FRAME_COUNT:
                return 900  # 30 সেকেন্ড
            return 0
        
        def isOpened(self):
            return True
        
        def release(self):
            pass
    
    return DummyVideoCapture()


if __name__ == "__main__":
    main()

### Final

football_analysis_complete/

├── main.py              # আপনার বর্তমান কোড (সংষ্কৃত)

├── pass_tracker.py      # নতুন: পাস ট্র্যাকিং লজিক

├── config.py            # নতুন: সেটিংস

├── utils.py             # নতুন: সাহায্যকারী ফাংশন

└── requirements.txt     # নতুন: লাইব্রেরি লিস্ট

In [ ]:
"""
config.py - সব সেটিংস এবং থ্রেশহোল্ড
"""

# থ্রেশহোল্ড সেটিংস
PASS_TIME_THRESHOLD = 1.0      # পাস হিসেবে গণ্য করার সর্বোচ্চ সময় (সেকেন্ড)
BALL_DISTANCE_THRESHOLD = 80   # বলের মালিক হবার সর্বোচ্চ দূরত্ব (পিক্সেল)
MIN_PASS_DISTANCE = 50         # পাসের ন্যূনতম দূরত্ব (পিক্সেল)
SUCCESSFUL_PASS_HOLD_TIME = 0.5  # সফল পাসের ন্যূনতম ধরে রাখার সময়

# কালার কোড (BGR ফরম্যাট)
COLORS = {
    'TEAM_A': (0, 0, 255),        # লাল
    'TEAM_B': (255, 0, 0),        # নীল
    'BALL': (0, 255, 255),        # হলুদ
    'QUICK_PASS': (0, 255, 0),    # সবুজ (দ্রুত পাস)
    'NORMAL_PASS': (255, 255, 0), # হলুদ (সাধারণ পাস)
    'SUCCESSFUL_PASS': (0, 255, 0), # সবুজ (সফল পাস)
    'FAILED_PASS': (0, 0, 255),   # লাল (ব্যর্থ পাস)
    'INTERCEPTION': (0, 165, 255) # কমলা (ইন্টারসেপশন)
}

# ভিজ্যুয়ালাইজেশন সেটিংস
SHOW_PASS_ARROWS = True
SHOW_POSSESSION_STATS = True
SHOW_PASS_HISTORY = True
MAX_PASS_HISTORY = 10

# দল সেটিংস (আপনার মডেল অনুযায়ী)
CLASS_IDS = {
    'BALL': 0,
    'GOALKEEPER': 1,
    'PLAYER': 2,
    'REFEREE': 3
}

TEAM_NAMES = {
    0: "Team Blue",
    1: "Team Pink"
}

In [ ]:
"""
utils.py - সাধারণ সাহায্যকারী ফাংশন
"""
import numpy as np
import cv2

def box_center(xyxy):
    """
    বাউন্ডিং বক্সের কেন্দ্র বের করে
    """
    x1, y1, x2, y2 = xyxy
    return np.array([(x1 + x2) / 2, (y1 + y2) / 2])

def calculate_distance(point1, point2):
    """
    দুটি পয়েন্টের মধ্যে ইউক্লিডীয় দূরত্ব বের করে
    """
    return np.linalg.norm(np.array(point1) - np.array(point2))

def get_player_team(player_class_id):
    """
    খেলোয়াড়ের দল নির্ধারণ করে
    """
    return "A" if player_class_id == 0 else "B"

def calculate_pass_type(duration, distance):
    """
    পাসের ধরন নির্ধারণ করে
    """
    if duration < 0.3:
        if distance > 200:
            return "QUICK_LONG_PASS"
        else:
            return "QUICK_SHORT_PASS"
    elif duration < 0.6:
        if distance > 200:
            return "NORMAL_LONG_PASS"
        else:
            return "NORMAL_SHORT_PASS"
    else:
        return "SLOW_PASS"

def draw_text_with_background(frame, text, position, font_scale=0.6, 
                             text_color=(255, 255, 255), bg_color=(0, 0, 0),
                             thickness=1):
    """
    ব্যাকগ্রাউন্ড সহ টেক্সট আঁকে
    """
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
    
    # ব্যাকগ্রাউন্ড রেকটেঙ্গেল
    bg_x1 = position[0] - 5
    bg_y1 = position[1] - text_size[1] - 5
    bg_x2 = position[0] + text_size[0] + 5
    bg_y2 = position[1] + 5
    
    cv2.rectangle(frame, (bg_x1, bg_y1), (bg_x2, bg_y2), bg_color, -1)
    
    # টেক্সট
    cv2.putText(frame, text, position, font, font_scale, text_color, thickness)
    
    return frame

In [ ]:
"""
pass_tracker.py - পাস এবং মালিকানা ট্র্যাকিং লজিক
"""
import numpy as np
from collections import defaultdict, deque
from config import *
from utils import *

class PassTracker:
    """পাস এবং বল মালিকানা ট্র্যাক করে"""
    
    def __init__(self, fps=30):
        self.fps = fps
        
        # বল মালিকানা ট্র্যাকিং
        self.current_owner = None
        self.current_owner_position = None
        self.current_owner_team = None
        self.ownership_start_time = 0
        self.ownership_start_frame = 0
        
        # পাস ইভেন্ট ট্র্যাকিং
        self.last_owner = None
        self.last_owner_position = None
        self.last_owner_team = None
        self.last_change_time = 0
        
        # পরিসংখ্যান
        self.pass_events = []  # সব পাস
        self.recent_passes = deque(maxlen=MAX_PASS_HISTORY)  # সাম্প্রতিক পাস
        
        self.possession_stats = {
            'team_a': {'frames': 0, 'time': 0.0},
            'team_b': {'frames': 0, 'time': 0.0},
            'no_possession': {'frames': 0, 'time': 0.0}
        }
        
        self.pass_stats = {
            'total': 0,
            'successful': 0,
            'team_a_internal': 0,
            'team_b_internal': 0,
            'interceptions': 0,
            'lost_balls': 0
        }
        
        # খেলোয়াড়ভিত্তিক পরিসংখ্যান
        self.player_stats = defaultdict(lambda: {
            'possession_frames': 0,
            'passes_made': 0,
            'passes_received': 0,
            'successful_passes': 0
        })
    
    def find_ball_owner(self, ball_detections, player_detections, frame_idx):
        """
        বলের বর্তমান মালিক খুঁজে বের করে
        """
        if len(ball_detections) == 0 or len(player_detections) == 0:
            return None, None, None
        
        # সবচেয়ে কনফিডেন্ট বল ডিটেকশন নিন
        if len(ball_detections) > 0:
            ball_box = ball_detections.xyxy[0]
            ball_center = box_center(ball_box)
        else:
            return None, None, None
        
        # সবচেয়ে কাছের খেলোয়াড় খুঁজুন
        min_distance = float('inf')
        owner_id = None
        owner_position = None
        owner_team = None
        
        for i, (box, tracker_id, class_id) in enumerate(zip(
            player_detections.xyxy, 
            player_detections.tracker_id, 
            player_detections.class_id
        )):
            if tracker_id is None:
                continue
            
            player_center = box_center(box)
            distance = calculate_distance(ball_center, player_center)
            
            if distance < min_distance and distance < BALL_DISTANCE_THRESHOLD:
                min_distance = distance
                owner_id = tracker_id
                owner_position = player_center
                owner_team = get_player_team(class_id)
        
        return owner_id, owner_position, owner_team
    
    def update_possession(self, owner_id, owner_position, owner_team, frame_idx, current_time):
        """
        মালিকানা আপডেট করে এবং পাস চেক করে
        """
        # প্রথম ফ্রেম বা মালিক নেই
        if self.current_owner is None and owner_id is None:
            self.current_owner = owner_id
            self.current_owner_position = owner_position
            self.current_owner_team = owner_team
            self.ownership_start_time = current_time
            self.ownership_start_frame = frame_idx
            return None
        
        # মালিক বদলালো
        if owner_id != self.current_owner:
            # পূর্ববর্তী মালিকের পরিসংখ্যান আপডেট
            if self.current_owner is not None:
                possession_duration = current_time - self.ownership_start_time
                self._update_player_possession_stats(
                    self.current_owner, possession_duration
                )
            
            # পাস ইভেন্ট চেক
            pass_event = self._check_pass_event(
                owner_id, owner_position, owner_team, 
                frame_idx, current_time
            )
            
            # নতুন মালিক সেট করুন
            self.current_owner = owner_id
            self.current_owner_position = owner_position
            self.current_owner_team = owner_team
            self.ownership_start_time = current_time
            self.ownership_start_frame = frame_idx
            
            return pass_event
        
        # মালিক একই আছে
        return None
    
    def _check_pass_event(self, new_owner, new_position, new_team, frame_idx, current_time):
        """
        পাস ইভেন্ট চেক করে
        """
        if self.last_owner is None or new_owner is None:
            # প্রথম বার বা বল কারো কাছে নেই
            self.last_owner = new_owner
            self.last_owner_position = new_position
            self.last_owner_team = new_team
            self.last_change_time = current_time
            return None
        
        # সময়ের পার্থক্য
        time_diff = current_time - self.last_change_time
        
        # পাস হয়েছে কিনা চেক
        if time_diff < PASS_TIME_THRESHOLD and new_owner != self.last_owner:
            # পাসের দূরত্ব
            pass_distance = calculate_distance(
                self.last_owner_position, new_position
            ) if self.last_owner_position is not None and new_position is not None else 0
            
            # পাসের ধরন
            pass_type = self._determine_pass_type(
                self.last_owner_team, new_team, time_diff, pass_distance
            )
            
            # পাস ইভেন্ট তৈরি
            pass_event = {
                'from_player': self.last_owner,
                'to_player': new_owner,
                'from_team': self.last_owner_team,
                'to_team': new_team,
                'from_position': self.last_owner_position,
                'to_position': new_position,
                'time': time_diff,
                'distance': pass_distance,
                'type': pass_type,
                'frame': frame_idx,
                'timestamp': current_time,
                'successful': self._is_pass_successful(time_diff, pass_distance)
            }
            
            # পরিসংখ্যান আপডেট
            self._update_pass_stats(pass_event)
            
            # খেলোয়াড় পরিসংখ্যান আপডেট
            self._update_player_pass_stats(pass_event)
            
            # ইতিহাসে যোগ
            self.pass_events.append(pass_event)
            self.recent_passes.append(pass_event)
            
            # আউটপুট প্রিন্ট
            self._print_pass_info(pass_event)
            
            # আপডেট
            self.last_owner = new_owner
            self.last_owner_position = new_position
            self.last_owner_team = new_team
            self.last_change_time = current_time
            
            return pass_event
        
        # আপডেট (পাস না হলেও)
        self.last_owner = new_owner
        self.last_owner_position = new_position
        self.last_owner_team = new_team
        self.last_change_time = current_time
        
        return None
    
    def _determine_pass_type(self, from_team, to_team, time_diff, distance):
        """পাসের ধরন নির্ধারণ"""
        # দলভিত্তিক
        if from_team == to_team:
            if from_team == "A":
                base_type = "TEAM_A_PASS"
            else:
                base_type = "TEAM_B_PASS"
        else:
            if from_team == "A" and to_team == "B":
                return "INTERCEPTION_BY_B"
            elif from_team == "B" and to_team == "A":
                return "INTERCEPTION_BY_A"
            else:
                base_type = "UNKNOWN_PASS"
        
        # সময়ভিত্তিক
        if time_diff < 0.3:
            time_type = "QUICK"
        elif time_diff < 0.6:
            time_type = "NORMAL"
        else:
            time_type = "SLOW"
        
        # দূরত্বভিত্তিক
        if distance > 200:
            dist_type = "LONG"
        else:
            dist_type = "SHORT"
        
        return f"{time_type}_{dist_type}_{base_type}"
    
    def _is_pass_successful(self, time_diff, distance):
        """পাস সফল কিনা চেক"""
        # সহজ লজিক: দ্রুত এবং মাঝারি দূরত্বের পাস সফল
        return time_diff < 0.5 and 50 < distance < 300
    
    def _update_pass_stats(self, pass_event):
        """পাস পরিসংখ্যান আপডেট"""
        self.pass_stats['total'] += 1
        
        if pass_event['successful']:
            self.pass_stats['successful'] += 1
        
        if pass_event['from_team'] == pass_event['to_team']:
            if pass_event['from_team'] == "A":
                self.pass_stats['team_a_internal'] += 1
            else:
                self.pass_stats['team_b_internal'] += 1
        else:
            self.pass_stats['interceptions'] += 1
    
    def _update_player_possession_stats(self, player_id, duration):
        """খেলোয়াড়ের মালিকানা পরিসংখ্যান আপডেট"""
        if player_id is not None:
            self.player_stats[player_id]['possession_frames'] += duration * self.fps
    
    def _update_player_pass_stats(self, pass_event):
        """খেলোয়াড়ের পাস পরিসংখ্যান আপডেট"""
        from_player = pass_event['from_player']
        to_player = pass_event['to_player']
        
        if from_player is not None:
            self.player_stats[from_player]['passes_made'] += 1
            if pass_event['successful']:
                self.player_stats[from_player]['successful_passes'] += 1
        
        if to_player is not None:
            self.player_stats[to_player]['passes_received'] += 1
    
    def _print_pass_info(self, pass_event):
        """পাস তথ্য প্রিন্ট"""
        icons = {
            "TEAM_A_PASS": "🔴",
            "TEAM_B_PASS": "🔵",
            "INTERCEPTION": "🟡",
            "QUICK": "⚡",
            "NORMAL": "➡️",
            "SLOW": "🐢"
        }
        
        # আইকন নির্বাচন
        icon = "⚽"
        for key in icons:
            if key in pass_event['type']:
                icon = icons[key]
                break
        
        print(f"{icon} PASS: Player {pass_event['from_player']} → Player {pass_event['to_player']}")
        print(f"   Type: {pass_event['type']}")
        print(f"   Time: {pass_event['time']:.3f}s | Distance: {pass_event['distance']:.1f}px")
        print(f"   Successful: {'✅' if pass_event['successful'] else '❌'}")
        print("-" * 40)
    
    def get_current_stats(self):
        """বর্তমান পরিসংখ্যান রিটার্ন করে"""
        total_frames = sum(team['frames'] for team in self.possession_stats.values())
        
        if total_frames == 0:
            team_a_percent = team_b_percent = 0
        else:
            team_a_percent = (self.possession_stats['team_a']['frames'] / total_frames) * 100
            team_b_percent = (self.possession_stats['team_b']['frames'] / total_frames) * 100
        
        # গড় পাস সময়
        avg_pass_time = 0
        if self.pass_stats['total'] > 0 and self.pass_events:
            avg_pass_time = sum(p['time'] for p in self.pass_events) / len(self.pass_events)
        
        # সফলতার হার
        success_rate = 0
        if self.pass_stats['total'] > 0:
            success_rate = (self.pass_stats['successful'] / self.pass_stats['total']) * 100
        
        return {
            'team_a_possession': f"{team_a_percent:.1f}%",
            'team_b_possession': f"{team_b_percent:.1f}%",
            'total_passes': self.pass_stats['total'],
            'successful_passes': self.pass_stats['successful'],
            'success_rate': f"{success_rate:.1f}%",
            'team_a_passes': self.pass_stats['team_a_internal'],
            'team_b_passes': self.pass_stats['team_b_internal'],
            'interceptions': self.pass_stats['interceptions'],
            'avg_pass_time': f"{avg_pass_time:.3f}s",
            'current_owner': self.current_owner,
            'current_owner_team': self.current_owner_team,
            'recent_passes': list(self.recent_passes)[-5:]  # শেষ ৫টি পাস
        }
    
    def print_summary(self):
        """সারাংশ প্রিন্ট"""
        stats = self.get_current_stats()
        
        print("\n" + "="*60)
        print("PASS ANALYSIS SUMMARY")
        print("="*60)
        
        print(f"\n📊 Possession:")
        print(f"  Team A: {stats['team_a_possession']}")
        print(f"  Team B: {stats['team_b_possession']}")
        
        print(f"\n🎯 Passing Statistics:")
        print(f"  Total Passes: {stats['total_passes']}")
        print(f"  Successful: {stats['successful_passes']}")
        print(f"  Success Rate: {stats['success_rate']}")
        print(f"  Team A Passes: {stats['team_a_passes']}")
        print(f"  Team B Passes: {stats['team_b_passes']}")
        print(f"  Interceptions: {stats['interceptions']}")
        print(f"  Avg Pass Time: {stats['avg_pass_time']}")
        
        print(f"\n👤 Player Statistics (Top 5):")
        sorted_players = sorted(
            self.player_stats.items(), 
            key=lambda x: x[1]['passes_made'], 
            reverse=True
        )[:5]
        
        for player_id, p_stats in sorted_players:
            if p_stats['passes_made'] > 0:
                success_rate = (p_stats['successful_passes'] / p_stats['passes_made']) * 100
                print(f"  Player {player_id}: {p_stats['passes_made']} passes, "
                      f"{p_stats['successful_passes']} successful ({success_rate:.1f}%)")
        
        print("="*60)

In [ ]:
"""
visualizer.py - ভিজ্যুয়ালাইজেশন ফাংশন
"""
import cv2
import numpy as np
from config import *
from utils import *

class FootballVisualizer:
    """ফুটবল বিশ্লেষণ ভিজ্যুয়ালাইজেশন"""
    
    def __init__(self, tracker):
        self.tracker = tracker
    
    def draw_ball_owner(self, frame, owner_id, owner_position, owner_team):
        """
        বল মালিক ভিজ্যুয়ালাইজ করে
        """
        if owner_id is None or owner_position is None:
            return frame
        
        x, y = int(owner_position[0]), int(owner_position[1])
        
        # মালিকের চারপাশে বৃত্ত
        color = COLORS['TEAM_A'] if owner_team == "A" else COLORS['TEAM_B']
        cv2.circle(frame, (x, y), 35, color, 3)
        
        # টেক্সট
        text = f"Owner: P{owner_id}"
        frame = draw_text_with_background(
            frame, text, (x - 40, y - 40),
            font_scale=0.6, text_color=(255, 255, 255), bg_color=color
        )
        
        return frame
    
    def draw_pass_arrow(self, frame, pass_event):
        """
        পাসের তীর আঁকে
        """
        if pass_event is None:
            return frame
        
        from_pos = pass_event['from_position']
        to_pos = pass_event['to_position']
        
        if from_pos is None or to_pos is None:
            return frame
        
        # কালার নির্ধারণ
        if pass_event['successful']:
            color = COLORS['SUCCESSFUL_PASS']
            thickness = 3
        else:
            color = COLORS['FAILED_PASS']
            thickness = 2
        
        # তীর আঁকা
        cv2.arrowedLine(
            frame,
            tuple(map(int, from_pos)),
            tuple(map(int, to_pos)),
            color,
            thickness,
            tipLength=0.2,
            line_type=cv2.LINE_AA
        )
        
        # পাসের তথ্য
        mid_x = int((from_pos[0] + to_pos[0]) / 2)
        mid_y = int((from_pos[1] + to_pos[1]) / 2)
        
        info_text = f"{pass_event['time']:.2f}s"
        frame = draw_text_with_background(
            frame, info_text, (mid_x - 20, mid_y - 15),
            font_scale=0.5, text_color=(255, 255, 255), bg_color=color
        )
        
        return frame
    
    def draw_stats_panel(self, frame, stats):
        """
        পরিসংখ্যান প্যানেল আঁকে
        """
        height, width = frame.shape[:2]
        
        # আধা-স্বচ্ছ প্যানেল
        overlay = frame.copy()
        cv2.rectangle(overlay, (10, 10), (350, 200), (0, 0, 0), -1)
        frame = cv2.addWeighted(overlay, 0.7, frame, 0.3, 0)
        
        # পরিসংখ্যান টেক্সট
        stats_text = [
            "⚽ LIVE PASS STATS",
            f"Team A: {stats.get('team_a_possession', '0%')}",
            f"Team B: {stats.get('team_b_possession', '0%')}",
            f"Total Passes: {stats.get('total_passes', 0)}",
            f"Successful: {stats.get('successful_passes', 0)}",
            f"Success Rate: {stats.get('success_rate', '0%')}",
            f"Avg Pass Time: {stats.get('avg_pass_time', '0s')}",
            f"Owner: P{stats.get('current_owner', 'None')}"
        ]
        
        # টেক্সট আঁকা
        y_offset = 40
        line_height = 22
        
        for i, text in enumerate(stats_text):
            color = (0, 255, 155) if i == 0 else (200, 200, 200)
            font_scale = 0.6 if i == 0 else 0.5
            
            cv2.putText(
                frame, text, (20, y_offset + i * line_height),
                cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, 1
            )
        
        return frame
    
    def draw_pass_history(self, frame, pass_history):
        """
        পাস ইতিহাস আঁকে
        """
        height, width = frame.shape[:2]
        
        # প্যানেল
        panel_x = width - 280
        cv2.rectangle(frame, (panel_x, 10), (width - 10, 200), (0, 0, 0), -1)
        
        # টাইটেল
        cv2.putText(
            frame, "RECENT PASSES", (panel_x + 10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1
        )
        
        # প্রতিটি পাস
        y_offset = 60
        recent_passes = pass_history[-5:] if pass_history else []
        
        for i, pass_event in enumerate(recent_passes):
            from_player = pass_event.get('from_player', '?')
            to_player = pass_event.get('to_player', '?')
            duration = pass_event.get('time', 0)
            successful = pass_event.get('successful', False)
            
            # কালার নির্ধারণ
            if successful:
                color = (0, 255, 0)  # সবুজ
            else:
                color = (255, 0, 0)  # লাল
            
            text = f"P{from_player}→P{to_player} ({duration:.2f}s)"
            
            cv2.putText(
                frame, text, (panel_x + 10, y_offset + i * 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1
            )
        
        return frame
    
    def draw_frame_info(self, frame, frame_idx, fps):
        """
        ফ্রেম তথ্য আঁকে
        """
        current_time = frame_idx / fps
        
        # সময় এবং ফ্রেম নম্বর
        time_text = f"Time: {current_time:.1f}s | Frame: {frame_idx}"
        cv2.putText(
            frame, time_text, (frame.shape[1] - 300, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2
        )
        
        return frame

In [ ]:
"""
main.py - আপনার মূল কোড + পাস ট্র্যাকিং
"""
import os
import cv2
import numpy as np
from tqdm import tqdm

import supervision as sv
from ultralytics import YOLO
from sports.common.team import TeamClassifier

# আমাদের নতুন মডিউল
from config import *
from foatball.pass_tracker import PassTracker
from visualizer import FootballVisualizer
from utils import *

# ============================================
# CONFIG
# ============================================
SOURCE_VIDEO_PATH = "input.mp4"          # change
OUTPUT_VIDEO_PATH = "output_with_passes.mp4"
MODEL_PATH = "foatball350.pt"             # change

TARGET_WIDTH = 1280
TARGET_HEIGHT = 720
CONF_THRES = 0.5

# ============================================
# INITIALIZE SYSTEMS
# ============================================
print("[INFO] Loading YOLO model...")
model = YOLO(MODEL_PATH)

# পাস ট্র্যাকার তৈরি
pass_tracker = PassTracker(fps=30)
visualizer = FootballVisualizer(pass_tracker)

# ============================================
# EXTRACT PLAYER CROPS (TEAM TRAINING)
# ============================================
def extract_player_crops(video_path, stride=20, max_crops=120):
    print("[INFO] Extracting player crops...")

    crops = []
    cap = cv2.VideoCapture(video_path)
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % stride != 0:
            frame_idx += 1
            continue

        frame = cv2.resize(frame, (TARGET_WIDTH, TARGET_HEIGHT))
        result = model.track(frame, persist=True, conf=0.3)[0]
        detections = sv.Detections.from_ultralytics(result)

        # class 2 = player
        players = detections[detections.class_id == 2]

        for box in players.xyxy:
            x1, y1, x2, y2 = map(int, box)
            crop = frame[y1:y2, x1:x2]
            if crop.size > 0:
                crops.append(crop)

        if len(crops) >= max_crops:
            break

        frame_idx += 1

    cap.release()
    print(f"[INFO] Collected {len(crops)} player crops")
    return crops

# ============================================
# TRAIN TEAM CLASSIFIER
# ============================================
player_crops = extract_player_crops(SOURCE_VIDEO_PATH)

print("[INFO] Training Team Classifier...")
team_classifier = TeamClassifier(device="cuda" if cv2.cuda.getCudaEnabledDeviceCount() else "cpu")
team_classifier.fit(player_crops)
print("[INFO] Team Classifier ready!")

# ============================================
# ANNOTATORS
# ============================================
team_palette = sv.ColorPalette.from_hex(["#00BFFF", "#FF1493"])

ellipse_annotator = sv.EllipseAnnotator(color=team_palette, thickness=2)
triangle_ball = sv.TriangleAnnotator(color=sv.Color.YELLOW)
box_ref = sv.BoxAnnotator(color=sv.Color.RED)
box_gk = sv.BoxAnnotator(color=sv.Color.GREEN)

label_player = sv.LabelAnnotator(
    color=team_palette,
    text_color=sv.Color.BLACK,
    text_scale=0.5
)

label_ref = sv.LabelAnnotator(color=sv.Color.RED, text_scale=0.4)
label_gk = sv.LabelAnnotator(color=sv.Color.GREEN, text_scale=0.4)

tracker = sv.ByteTrack()

# ============================================
# VIDEO WRITER
# ============================================
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS) or 30
cap.release()

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(
    OUTPUT_VIDEO_PATH,
    fourcc,
    fps,
    (TARGET_WIDTH, TARGET_HEIGHT)
)

# ============================================
# MAIN LOOP WITH PASS TRACKING
# ============================================
print("[INFO] Processing video with pass tracking...")
frame_gen = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

# পাস ইভেন্ট স্টোর করতে
all_pass_events = []

for frame_idx, frame in enumerate(tqdm(frame_gen)):
    frame = cv2.resize(frame, (TARGET_WIDTH, TARGET_HEIGHT))
    annotated = frame.copy()
    
    try:
        result = model.track(frame, persist=True, conf=CONF_THRES)[0]
        detections = sv.Detections.from_ultralytics(result)

        ball = detections[detections.class_id == 0]
        goalkeeper = detections[detections.class_id == 1]
        players = detections[detections.class_id == 2]
        referee = detections[detections.class_id == 3]

        # NMS + Tracking
        players = players.with_nms(0.3)
        players = tracker.update_with_detections(players)

        # ===== TEAM CLASSIFICATION =====
        crops, valid_ids = [], []
        for i, box in enumerate(players.xyxy):
            x1, y1, x2, y2 = map(int, box)
            crop = frame[y1:y2, x1:x2]
            if crop.size > 0:
                crops.append(crop)
                valid_ids.append(i)

        team_ids = [0] * len(players)
        if crops:
            preds = team_classifier.predict(crops)
            for idx, team_id in zip(valid_ids, preds):
                team_ids[idx] = team_id

        players.class_id = team_ids

        # ===== BALL OWNER DETECTION =====
        owner_id, owner_position, owner_team = pass_tracker.find_ball_owner(
            ball, players, frame_idx
        )
        
        current_time = frame_idx / fps
        
        # ===== PASS DETECTION =====
        pass_event = pass_tracker.update_possession(
            owner_id, owner_position, owner_team, frame_idx, current_time
        )
        
        if pass_event:
            all_pass_events.append(pass_event)
        
        # ===== GET CURRENT STATS =====
        current_stats = pass_tracker.get_current_stats()
        
        # ===== LABELS =====
        labels = []
        for tid, pid in zip(players.class_id, players.tracker_id):
            labels.append(f"T{tid+1}-{pid}")

        # ===== DRAW ORIGINAL ANNOTATIONS =====
        if len(players):
            annotated = ellipse_annotator.annotate(annotated, players)
            annotated = label_player.annotate(annotated, players, labels)

        if len(ball):
            annotated = triangle_ball.annotate(annotated, ball)

        if len(goalkeeper):
            annotated = box_gk.annotate(annotated, goalkeeper)
            annotated = label_gk.annotate(
                annotated, goalkeeper,
                [f"GK {c:.2f}" for c in goalkeeper.confidence]
            )

        if len(referee):
            annotated = box_ref.annotate(annotated, referee)
            annotated = label_ref.annotate(
                annotated, referee,
                [f"REF {c:.2f}" for c in referee.confidence]
            )
        
        # ===== DRAW PASS VISUALIZATIONS =====
        
        # ১. বল মালিক দেখানো
        if owner_id is not None:
            annotated = visualizer.draw_ball_owner(
                annotated, owner_id, owner_position, owner_team
            )
        
        # ২. পাস তীর আঁকা (যদি পাস হয়)
        if pass_event and SHOW_PASS_ARROWS:
            annotated = visualizer.draw_pass_arrow(annotated, pass_event)
        
        # ৩. পরিসংখ্যান প্যানেল
        if SHOW_POSSESSION_STATS:
            annotated = visualizer.draw_stats_panel(annotated, current_stats)
        
        # ৪. পাস ইতিহাস
        if SHOW_PASS_HISTORY:
            annotated = visualizer.draw_pass_history(
                annotated, current_stats.get('recent_passes', [])
            )
        
        # ৫. ফ্রেম তথ্য
        annotated = visualizer.draw_frame_info(annotated, frame_idx, fps)
        
        # ৬. ফ্রেম নম্বর
        cv2.putText(
            annotated, f"Frame {frame_idx}",
            (20, 40), cv2.FONT_HERSHEY_SIMPLEX,
            0.7, (255, 255, 255), 2
        )

    except Exception as e:
        print(f"[WARN] Frame {frame_idx} error: {e}")
        annotated = frame

    out.write(annotated)

# ============================================
# FINALIZE
# ============================================
out.release()

print("\n" + "="*60)
print("PROCESSING COMPLETE!")
print("="*60)

# ফাইনাল রিপোর্ট
pass_tracker.print_summary()

print(f"\n📊 Total Pass Events Detected: {len(all_pass_events)}")
print(f"📁 Output saved to: {OUTPUT_VIDEO_PATH}")

# পাস ডেটা সেভ (ঐচ্ছিক)
if all_pass_events:
    import json
    with open('pass_events.json', 'w') as f:
        json.dump(all_pass_events, f, indent=2)
    print("💾 Pass events saved to pass_events.json")

print("="*60)